In [ ]:
#Download data through kaggle CLI
#(After dowloading the access token from kaggle (make sure your phone and identity have been verified in your profile)
# kaggle datasets download -d Cornell-University/arxiv -p destination_folder --unzip

In [21]:
import os
import sys

#Import config file. Update config.py according to your environment
import config

import pandas as pd

import requests, json

## 1. Load dataset

In [5]:
data_path = os.path.join(config.path_to_data, 'arxiv-metadata-oai-snapshot.json')
data = pd.read_json(data_path, lines=True)

In [7]:
data.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [34]:
r = requests.post(
    'https://api.semanticscholar.org/graph/v1/paper/batch',
    params={'fields': 'citationCount'},
    json={"ids": doi_names[:500]}
)
print(json.dumps(r.json(), indent=2))

[
  {
    "paperId": "08d86345c6a4b1e27dc74ebd89b942c56faaa7a3",
    "citationCount": 47
  },
  {
    "paperId": "7084716fffb2382084addf934ead6b8a37c74c43",
    "citationCount": 7
  },
  {
    "paperId": "596e5276e4d36c6f4891d0c286f55ce79e682c50",
    "citationCount": 153
  },
  {
    "paperId": "a4db7ebeac15458f907a2f05345f4c135ccbde03",
    "citationCount": 10
  },
  {
    "paperId": "5ebee094c149e16f1a90f4c1b3382ea32e13906e",
    "citationCount": 160
  },
  {
    "paperId": "40a87a0bcd282e66980aa13a25171b91390fca47",
    "citationCount": 16
  },
  {
    "paperId": "223f376ab747118816f707e6b9d5bea772fa1743",
    "citationCount": 0
  },
  {
    "paperId": "e33b112473bff21cc7eed7a60553313d0f2a0781",
    "citationCount": 13
  },
  {
    "paperId": "4e2e756d24cb12ca9eda61266b557bd419e262ed",
    "citationCount": 1
  },
  {
    "paperId": "5175b594ae9295f679dc42688ee0e71fa4474072",
    "citationCount": 10
  },
  null,
  {
    "paperId": "8000fe895171e34b71c75c4929afe135a53e4f1c",
    "cit